In [1]:
!pip install llama-cpp-python


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import hf_hub_download
import os

model_repo = "gaianet/Qwen2.5-7B-Instruct-GGUF"
model_basename = "Qwen2.5-7B-Instruct-Q4_K_M.gguf"

model_path = "../models/" + model_basename

if not os.path.exists(model_path):
    model_path = hf_hub_download(
        repo_id=model_repo,
        filename=model_basename,
        local_dir="./models",
    )
else:
    print(f"Model {model_basename} is already downloaded.")

Model Qwen2.5-7B-Instruct-Q4_K_M.gguf is already downloaded.


In [3]:
from langchain_community.chat_models import ChatLlamaCpp
from langchain_community.embeddings import LlamaCppEmbeddings

llm = ChatLlamaCpp(
    model_path=model_path,
    temperature=0.1,
    max_tokens=1024,
    n_ctx=32768,
    n_gpu_layers=40,
    n_batch=512,
    verbose=False,
)


In [4]:
test_response = llm.invoke("Say 'Hello!'")
print(test_response.content)

Hello! How can I assist you today?


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import Runnable

In [6]:
template_string = """<s>[INST] Ты являешься помошником. Отвечай на запросы пользователя используя *только* представленный ниже контекст. Не используй никакой иной информации. Если ответа в контесте нет, то говори, что ты не знаешь

Контекст: {context}

Запрос: {user_query} [/INST]"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [7]:
user_query = "Что такое RAG?"

context = "Генерация, дополненная поиском (Retrieval-Augmented Generation, RAG) — это подход, при котором генерация ответа большой языковой модели (LLM) осуществляется на основе данных, полученных в результате поиска во внешних источниках (файлы, базы данных, Интернет и другие источники). RAG-система работает в два основных этапа: сначала происходит извлечение релевантных документов или их частей из внешней базы знаний на основе запроса пользователя, а затем полученная информация подставляется вместе со специальными подсказками, указывающими как модель должна использовать эти данные, в контекст языковой модели для генерации итогового ответа. В зависимости от указаний в подсказках, сгенерированный ответ может включать цитаты или ссылки на исходные документы, что повышает прозрачность и доверие пользователей, позволяя проверить информацию. RAG помогает устранить ограничения LLM, такие как устаревание информации, наличие неточностей или появления галлюцинаций[1][2]."

In [8]:
formatted_prompt_message = prompt_template.format_messages(user_query=user_query, context=context)

print(llm.invoke(formatted_prompt_message).content)

RAG — это сокращение от "Retrieval-Augmented Generation". Этот подход использует генерацию ответа большой языковой модели (LLM), но добавляет к этому процессу данные, полученные в результате поиска во внешних источниках.
